In [1]:
import jax

jax.config.update("jax_platform_name", "cpu")

In [2]:
import numpy as np
import scanpy as sc

In [3]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [5]:
decoded_ood = np.matmul(adata_ood.obsm["X_pca"], adata_train.varm["PCs"].T) + adata_ood.varm["X_train_mean"].T

In [6]:
from ott.geometry import costs, pointcloud

cm_decoded = pointcloud.PointCloud(x=decoded_ood, y=decoded_ood, cost_fn=costs.Euclidean()).cost_matrix

cm_orig = pointcloud.PointCloud(x=adata_ood.X.A, y=adata_ood.X.A, cost_fn=costs.Euclidean()).cost_matrix

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory allocating 1269878408000 bytes.

In [ ]:
np.corrcoef(cm_decoded.flatten(), cm_orig.flatten())

In [12]:
np.corrcoef(cm_decoded.flatten(), cm_orig.flatten())

array([[1.        , 0.83154561],
       [0.83154561, 1.        ]])

In [13]:
from sklearn.metrics import r2_score


def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [14]:
compute_r_squared(decoded_ood, adata_ood.X.A)

0.9994916136435403